Mayra Diandra Nabila Ratnadi (3039839)

Michelle Espranita Liman (3072994)

For this task, we assumed that the spelling corrector is implemented for writing mistakes, since the test set given is a handwritten essay. Thus, we take no typing mistakes into account.

In [1]:
import numpy as np
from nltk import bigrams
from nltk import FreqDist
from nltk.tokenize import word_tokenize

In [8]:
import operator
import string
from collections import Counter

In [9]:
import re
import pandas as pd

## Language model

In [31]:
def unigramFDist():
    with open("deu_news_2015_30K-sentences.txt", mode="r", encoding="utf-8") as f:
        corpus = f.read()
    corpus = re.findall(r'\w+', corpus)
    corpus = [word for word in corpus if not any([letter in ['0','1','2','3','4','5','6','7','8','9'] for letter in word])]
    # corpus is a list of words
    fdist = FreqDist(corpus)
    return fdist

In [32]:
def bigramFDist():
    with open("deu_news_2015_30K-sentences.txt", mode="r", encoding="utf-8") as f:
        corpus = f.read().splitlines()
    sentences = [word_tokenize(sentence) for sentence in corpus]
    sentences = [sentence[2:] for sentence in sentences]
    bi = [bigrams(sentence) for sentence in sentences]
    bi = [bigram for sublist in bi for bigram in sublist]
    bi_fdist = FreqDist(bi)
    return bi_fdist

In [33]:
fdist = unigramFDist()

In [34]:
bi_fdist = bigramFDist()

In [35]:
spellingErrors = pd.read_csv("spellingErrors.csv", sep='\t', names=['wrong', 'right'])

## Candidate Model

In [36]:
def oneEditAway(word):
    letters    = 'abcdefghijklmnopqrstuvwxyzäüöß'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    capitalize = [word.capitalize()]
    return set(deletes + transposes + replaces + inserts + capitalize)

In [37]:
def twoEditsAway(word):
    return set(e2 for e1 in oneEditAway(word) for e2 in oneEditAway(e1))

In [38]:
def candidates(word):
    return oneEditAway(word) | twoEditsAway(word)

In [39]:
def knownCandidates(word):
    cands = candidates(word)
    return [cand for cand in cands if fdist[cand]>0]

In [40]:
def cognitiveCandidates(word):
    cands = []
    for char in word:
        if char in list(spellingErrors["wrong"]):
            replacement = spellingErrors.loc[spellingErrors['wrong']==char]['right'].values[0]
            idx = word.find(char)
            cands.append(word[:idx] + replacement + word[(idx+1):])
    for i in range(len(word)-1):
        letters = word[i] + word[i+1]
        if letters in list(spellingErrors["wrong"]):
            replacement = spellingErrors.loc[spellingErrors['wrong']==letters]['right'].values[0]
            idx = word.find(letters)
            cands.append(word[:idx] + replacement + word[(idx+2):])
    return set(cands)

In [41]:
def cognitiveCandidates2(word):
    return set(c2 for c1 in cognitiveCandidates(word) for c2 in cognitiveCandidates(c1))

In [42]:
def totalCognitiveCandidates(word):
    return list(cognitiveCandidates(word) | cognitiveCandidates2(word))

In [43]:
def hCandidates(word):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    splits = [(L, R) for (L, R) in splits if len(L)!=0]
    hCands = [L + 'h' + R for (L,R) in splits]
    scores = {cand: fdist.freq(cand) for cand in hCands if fdist.freq(cand) > 0}
    filteredHCands = list(scores.keys())
    return filteredHCands

In [44]:
def repeatedCandidates(word):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    splits = [(L, R) for (L, R) in splits if len(L)!=0]
    repeatedCands = [L + L[-1] + R for (L,R) in splits]
    scores = {cand: fdist.freq(cand) for cand in repeatedCands if fdist.freq(cand) > 0}
    filteredRepeatedCands = list(scores.keys())
    return filteredRepeatedCands

In [45]:
def missingCandidates(word):
    missingCands = []
    for char in word:
        idx = word.find(char)
        missingCands.append(word[:idx] + word[(idx+1):])
    missingCands = list(set(missingCands))
    scores = {cand: fdist.freq(cand) for cand in missingCands if fdist.freq(cand) > 0}
    filteredMissingCands = list(scores.keys())
    return filteredMissingCands

## Error Model

In [46]:
def levenshtein(seq1, seq2):
    
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    
    matrix = np.zeros ((size_x, size_y))
    
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y
    
        
    for x in range(1, size_x):
        for y in range(1, size_y):
            
            if seq1[x-1] == seq2[y-1]:
                cost = 0
            
            ### cost for only changing letter case is set to 0.5
            elif seq1[x-1].lower() == seq2[y-1].lower():
                cost = 0.5
            
            else:
                cost = 1
                
            matrix [x,y] = min(
                matrix[x-1,y] + 1,
                matrix[x-1,y-1] + cost,
                matrix[x,y-1] + 1
                )
            
    #print (matrix)
    return (matrix[size_x - 1, size_y - 1])

In [47]:
def correction(word, prev):
    if fdist.freq(word)>0 or word in string.punctuation: # If it exists in the dataset, most probably it is already correct (no need to be corrected)
        return word
    else:
        # 1: Missing letters that are repeated (This is more probable, so we will test this first)
        repeatedCands = repeatedCandidates(word)
        if len(repeatedCands) == 1:
            return repeatedCands[0]
        # 2: Too many letters are repeated
        missingCands = missingCandidates(word)
        if len(missingCands) == 1:
            return missingCands[0]
        # 3: Missing 'h's (Because they are often silent in German)
        hCands = hCandidates(word)
        if len(hCands) == 1:
            return hCands[0]
        # 4: Capitalization error
        if fdist.freq(word.capitalize())>0:
            return word.capitalize()
        # 5: Pronounciation (Cognitive) error and Real Word error
        knownCands = knownCandidates(word)
        cognitiveCands = totalCognitiveCandidates(word)
        allCands = knownCands + cognitiveCands
        scores = {cand: ((len(word) - levenshtein(word, cand))/len(word)) * fdist.freq(cand) for cand in allCands}
        if all([score==0.0 for (word, score) in list(scores.items())]): # If all the candidates cannot be found in the dataset (Dataset insufficient/The error is too much)
            return word
        else: # Consider bigrams with the word before to bring some context, thus better judgment
            fiveMostLikelyScore = Counter(scores).most_common(5)
            fiveMostLikely = [scorer for (scorer, score) in fiveMostLikelyScore]
            bigrams = [(prev, item) for item in fiveMostLikely]
            scores = {big: bi_fdist.freq(big) + score for big in bigrams for (scorer, score) in fiveMostLikelyScore if big[1] == scorer}
            return Counter(scores).most_common(1)[0][0][1]

##  Complete Model

In [48]:
def correctText(listOfTokens):
    fdist = unigramFDist()
    corrections = []
    listOfTokens.insert(0, '')
    for i in range(1, len(listOfTokens)):
        prev = listOfTokens[i-1]
        word = listOfTokens[i]
        if word == '':
            continue
        print("Original: ", word)
        correctWord = correction(word, prev)
        listOfTokens[i] = correctWord
        print("Corrected: ", correctWord)
        corrections.append(correctWord)
    listOfTokens.pop(0)
    return corrections

In [49]:
def evaluate(corrected_tokens, original_spellings, solution):
    
    #make sure that all three lists have the same length
    assert len(corrected_tokens) == len(original_spellings) == len(solution), "Error. The lists with proposed corrections, original tokens and the solution do not have the same lengths."
    
    #count number of incorrect tokens (=whenever there is a difference between the proposed correction and the gold solution)
    incorrect_tokens = sum(1 for i in range(len(corrected_tokens)) if corrected_tokens[i] != solution[i])
    
    print("Number of incorrect tokens in the text:", incorrect_tokens)
    

def correct_and_evaluate(filename):
    
    #read in text
    with open(filename, mode="r", encoding="utf-8") as f:
        text = f.read().splitlines()
    text = [line.split("\t") for line in text]

    #extract original spelling and solution for every word
    original_spellings = [line[0] for line in text]
    solution = [line[1] for line in text]

    #correct tokens
    proposed_corrections = correctText(original_spellings)
    
    #evaluate corrections
    evaluate(proposed_corrections, original_spellings, solution)

In [50]:
correct_and_evaluate("spelling_homework.csv")

Original:  Lars
Corrected:  Lars
Original:  und
Corrected:  und
Original:  Lea
Corrected:  Lea
Original:  kaufen
Corrected:  kaufen
Original:  ein
Corrected:  ein
Original:  eis
Corrected:  Eis
Original:  .
Corrected:  .
Original:  Der
Corrected:  Der
Original:  eisferkeufer
Corrected:  eisferkeufer
Original:  gibt
Corrected:  gibt
Original:  in
Corrected:  in
Original:  eins
Corrected:  eins
Original:  .
Corrected:  .
Original:  Dan
Corrected:  Dan
Original:  gen
Corrected:  gen
Original:  sie
Corrected:  sie
Original:  und
Corrected:  und
Original:  esen
Corrected:  essen
Original:  ir
Corrected:  ihr
Original:  eis
Corrected:  Eis
Original:  .
Corrected:  .
Original:  Dodo
Corrected:  Doch
Original:  fenkt
Corrected:  fest
Original:  die
Corrected:  die
Original:  tropfen
Corrected:  Tropfen
Original:  die
Corrected:  die
Original:  runter
Corrected:  runter
Original:  fallen
Corrected:  fallen
Original:  .
Corrected:  .
Original:  Dodo
Corrected:  Doch
Original:  get
Corrected:  ge